In [55]:
import os
import time
import pickle
import numpy as np
import networkx as nx

from scipy import signal
from scipy.fft import fft, fftfreq, fftshift
from scipy.signal import find_peaks, butter, filtfilt
from scipy.ndimage import gaussian_filter
from scipy.io import wavfile

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
# read pickle file

with open("../data/processed_data/slices_and_features.pkl", "rb") as f:
    data_slices, slice_annotations, features = pickle.load(f)

In [3]:
# plot 10 random slices with their respective annotations

rand_ixs = np.random.randint(0, len(features), size=10)
fig = make_subplots(rows=5, cols=2, subplot_titles=[f"Slice {i}" for i in range(10)])
for i, ix in enumerate(rand_ixs):
    color = 'red' if slice_annotations[ix] == 1 else 'blue'
    fig.add_trace(
        go.Scatter(
            y=features[ix],
            mode='lines',
            name=f"Slice {ix}, Annotation: {slice_annotations[ix]}",
            line=dict(color=color)
        ),
        row=i//2 + 1,
        col=i%2 + 1
    )
fig.update_layout(height=1500, width=800, title_text="Features of Random Slices with Annotations")
fig.show()


In [53]:
# plot a random slice and its annotation

rand_ix = np.random.randint(0, len(features))
color = 'red' if slice_annotations[rand_ix] == 1 else 'blue'
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        y=features[rand_ix],
        mode='lines',
        name=f"Slice {rand_ix}, Annotation: {slice_annotations[rand_ix]}",
        line=dict(color=color)
    )
)
fig.update_layout(height=600, width=800, title_text="Feature of a Random Slice with Annotation")
fig.show()

---

In [57]:
# train a random forest classifier on the features and annotations
X_train, X_test, y_train, y_test = train_test_split(features, slice_annotations, test_size=0.2, random_state=42)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.57      0.27      0.36        15
           1       0.56      0.82      0.67        17

    accuracy                           0.56        32
   macro avg       0.57      0.55      0.52        32
weighted avg       0.57      0.56      0.52        32

[[ 4 11]
 [ 3 14]]


In [58]:
# train a simple SVM classifier on the features and annotations
from sklearn.svm import SVC
svm_clf = SVC(kernel='linear', random_state=42)
svm_clf.fit(X_train, y_train)
y_pred_svm = svm_clf.predict(X_test)
print(classification_report(y_test, y_pred_svm))
print(confusion_matrix(y_test, y_pred_svm))

              precision    recall  f1-score   support

           0       0.60      0.40      0.48        15
           1       0.59      0.76      0.67        17

    accuracy                           0.59        32
   macro avg       0.60      0.58      0.57        32
weighted avg       0.60      0.59      0.58        32

[[ 6  9]
 [ 4 13]]
